<a href="https://colab.research.google.com/github/Anirudh-gupta-g/Water-navigation-Agent/blob/main/Water_Navigation_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem solving by Uninformed & Informed Search

### 1.	Define the environment in the following block

Defining PEAS of the Agent

1.Performance Measure:
Minimize the total distance or time taken to reach the sea from a designated start location.  
Maximize the efficiency of inspecting and maintaining the entire network by visiting every location at least once with the least redundancy.  
Minimize the number of gates opened to the sea to prevent flooding.

2.Environment:
The water channel network in Chennai, represented as a graph with locations (nodes) and channels (edges).
Channels are unidirectional, allowing water flow in one direction.
Each location has gates that can be open or closed to regulate water flow.
The environment includes various start locations and a single goal location (the sea).

3.Actuators:
Gate open/close mechanism at each location to regulate water flow.
The agent's decision-making process to choose the next location to move towards the sea or to inspect and maintain the network.

4.Sensors:
Sensors to detect the current location of the agent within the network.
Water level sensors at each location to assess flood risk.
Sensors to detect the status (open or closed) of gates at each location.


Design the agent as PSA Agent(Problem Solving Agent)

1.Goal Formulation: The agent's goal is to find the shortest and most efficient path from a given start location to the sea, ensuring flood prevention by regulating gate operations and to plan a route that covers the entire network for inspection and maintenance purposes.

2.Problem Formulation: The problem can be formulated as a pathfinding problem in a directed graph, where the agent must decide at each step which channel to take next based on the current state of the environment, the performance measure, and the ultimate goal of reaching the sea or covering the entire network.

3.Search and Planning: The agent can use informed search algorithms like A* for finding the shortest path to the sea and local search algorithms like Random restart hill climbing for planning an efficient inspection and maintenance route.

4.Execution: The agent executes the chosen path by opening and closing gates as necessary to navigate through the network towards the sea or to cover the entire network for inspection


In [ ]:
#Setting Initial State
def set_initial_state(node):
    initial_state = node
    return initial_state

In [ ]:
#Code Block : Setting the transistion Matrix or Adjacency list
class PriorityQueue:
    def __init__(self):
        self.queue = []

    def put(self, item):
        self.queue.append(item)
        self.queue.sort()

    def get(self):
        return self.queue.pop(0)


class Graph:
    def __init__(self):
        self.edges = {}

    def add_weighted_edges(self, weighted_edges):
        for edge in weighted_edges:
            self.add_weighted_edge(edge)

    def add_weighted_edge(self, edge):
        start, end, weight = edge
        if start not in self.edges:
            self.edges[start] = []
        self.edges[start].append((end, weight))

    def nodes(self):
        return list(self.edges.keys())

    def neighbors(self, node):
        return [neighbor[0] for neighbor in self.edges.get(node, [])]

    def get_edge_weight(self, start, end):
        for neighbor, weight in self.edges.get(start, []):
            if neighbor == end:
                return weight
        return float('inf')


def degree_heuristic(node, goal, graph):
    return len(graph.neighbors(node))

graph = Graph()
graph.add_weighted_edges([('A', 'B', 2), ('A', 'C', 1), ('B', 'F', 1), ('B', 'G', 1), ('C', 'D', 2),
                          ('C', 'E', 2), ('D', 'E', 1), ('D', 'K', 1), ('E', 'F', 3), ('E', 'S', 2),
                          ('F', 'G', 1), ('G', 'H', 1), ('H', 'S', 1), ('I', 'A', 2), ('J', 'C', 1),
                          ('K', 'E', 1)])

In [ ]:
#Code Block :function to design the Transition Model/Successor function.
def successor_function(graph, node):
    return graph.neighbors(node)

In [ ]:
#Code block :fucntion to handle goal test (Must handle dynamic inputs).
def is_goal_state(node, goal):
    return node == goal

### A* Algorithm

In [ ]:
#Code Block : Function for A* algorithm implementation
import time
import sys

def a_star(graph, start, goal):

    start_time = time.time()

    open_set = PriorityQueue()
    open_set.put((0, start))
    came_from = {}
    path_cost = {node: float('inf') for node in graph.nodes()}
    path_cost[start] = 0
    path_cost[goal] = float('inf')
    fitness_score = {node: float('inf') for node in graph.nodes()}
    fitness_score[start] = degree_heuristic(start, goal, graph)

    space_complexity = sys.getsizeof(open_set) + sys.getsizeof(came_from) + sys.getsizeof(path_cost) + sys.getsizeof(fitness_score)
    iterations = 0

    while open_set.queue:
        current = open_set.get()[1]
        iterations += 1

        if is_goal_state(current, goal):
            path = []
            total_cost = 0
            while current in came_from:
                path.append(current)
                total_cost += graph.get_edge_weight(came_from[current], current)
                current = came_from[current]
            path.append(start)
            space_complexity += sys.getsizeof(path)

            elapsed_time = time.time() - start_time

            return path[::-1], total_cost, elapsed_time , iterations, space_complexity

        for neighbor in successor_function(graph, current):
            tentative_path_cost = path_cost[current] + graph.get_edge_weight(current, neighbor)
            if tentative_path_cost < path_cost[neighbor]:
                came_from[neighbor] = current
                path_cost[neighbor] = tentative_path_cost
                fitness_score[neighbor] = path_cost[neighbor] + degree_heuristic(neighbor, goal, graph)
                open_set.put((fitness_score[neighbor], neighbor))

    elapsed_time = time.time()- start_time

    return None, 0, elapsed_time, iterations, space_complexity


### Random restart hill climbing Algorithm

In [ ]:
#Code Block : Function for random restart hill climbing algorithm implementation
import random

def random_restart_hill_climbing(graph, start, goal, restarts=100):
    start_time = time.time()
    space_complexity = sys.getsizeof(graph) + sys.getsizeof(start) + sys.getsizeof(goal)

    all_nodes = graph.nodes()
    best_path = get_random_path(graph, start, goal, all_nodes)

    for _ in range(restarts):
        new_path = get_random_path(graph, start, goal, all_nodes)

        if len(new_path) < len(best_path) and new_path[-1] == goal:
            best_path = new_path

    space_complexity += sys.getsizeof(best_path)
    total_elapsed_time = time.time() - start_time

    return best_path, total_elapsed_time, space_complexity

def get_random_path(graph, start, goal, all_nodes):
    path = [start]
    current = start

    remaining_nodes = all_nodes.copy()
    remaining_nodes.remove(start)

    while remaining_nodes:
        neighbor_fitness = {
            node: len(path + [node]) + degree_heuristic_value(node, graph)
            for node in remaining_nodes
        }

        sorted_neighbors = sorted(neighbor_fitness, key=neighbor_fitness.get)
        next_node = sorted_neighbors[0]

        path.append(next_node)
        current = next_node
        remaining_nodes.remove(next_node)

    return path

def degree_heuristic_value(node, graph):
    return len(graph.neighbors(node))


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [ ]:
#Code Block : Function & call to get inputs (start/end state)
start_node = set_initial_state('I')
goal_node = 'S'  # Destination node (Sea)

### Calling the search algorithms

In [ ]:

#Invoking A* Algorithm
path, total_cost, elapsed_time, iterations, space_complexity = a_star(graph, start_node, goal_node)
print(f"A* Algorithm Path: {path}")
print(f"Total Cost: {total_cost}")

A* Algorithm Path: ['I', 'A', 'C', 'E', 'S']
Total Cost: 7


In [ ]:
# Invoking Random Restart Hill Climbing Algorithm
path, elapsed_time, space_complexity = random_restart_hill_climbing(graph, start_node, goal_node)

print(f"Random Restart Hill Climbing Algorithm Path: {path}")

Random Restart Hill Climbing Algorithm Path: ['I', 'F', 'G', 'H', 'J', 'K', 'A', 'B', 'C', 'D', 'E']


In [ ]:
#Code Block : Printing Time & Space complexity of A* algorithm
def print_time_and_space_complexity(algorithm_name, path, elapsed_time, iterations, space_complexity):
    print(f'Time complexity for {algorithm_name}: {elapsed_time:.6f} seconds')
    print(f'space complexity for {algorithm_name}: {space_complexity} bytes')

path, total_cost, elapsed_time, iterations, space_complexity = a_star(graph, start_node, goal_node)
print_time_and_space_complexity("A*", path, elapsed_time, iterations, space_complexity)

Time complexity for A*: 0.000153 seconds
space complexity for A*: 1512 bytes


In [ ]:
#Code Block : Printing Time & Space complexity of random restart hill climbing algorithm
def print_time_and_space_complexity(algorithm_name, path, elapsed_time, iterations, space_complexity):
    print(f'Time complexity for {algorithm_name}: {elapsed_time:.6f} seconds')
    print(f'Space complexity for {algorithm_name}: {space_complexity} bytes')

path, elapsed_time, space_complexity = random_restart_hill_climbing(graph, start_node, goal_node)
print_time_and_space_complexity("random_restart_hill_climbing", path, elapsed_time, iterations, space_complexity)

Time complexity for random_restart_hill_climbing: 0.011057 seconds
Space complexity for random_restart_hill_climbing: 332 bytes


bold text### 6.comparitive analysis or findings in no more than 3 lines in below section

**Comparison** :
1. Memory space usage of A* Algorithm is Significanty large compared to Random restart hill climbing Algorithm
2.Time complexity of A* Algorithm is Significantly small compared to Random restart hill climbing Algorithm